# Create a cohort calendar

**Input**
- Use the <a href = "https://docs.google.com/spreadsheets/d/1BZVS8D3Z0jkZxUeQNlGvwn88P9nH2ZM2j8WzdkEC4kc/edit?usp=sharing">Master Cohort Calendar</a> to confirm:
    - Enter in cohort name
    - Enter in start and stop dates
    - Update `student_holidays.csv`
- Enter in Student PD days (with Placement) if known; if not known, enter `NONE`
- Enter the filename of lessons days to utilize


**Output**

- student facing calendar (for syllabus)
- cohort-specific holidays (for syllabus)
- "cohort_cal.csv" (for instructors google calendar)

#### Enter in details below

In [1]:
cohort_name = 'ursula'

start_date = '10/03/2023'
end_date = '03/18/2024'

pd_day1 = None
pd_day2 = None

lesson_file = 'generic_lessons_days_99.csv'

#### Run following cells to build  lesson days

In [2]:
#imports
import pandas as pd
import numpy as np

from datetime import datetime, timedelta

In [6]:
#turn holidays into a df from csv
holiday_df = pd.read_csv('student_holidays.csv')
holiday_df.columns = ['date','class_type']
holiday_df.date = holiday_df.date.astype('datetime64[ns]')

#turn pd days into df from inputs
pd_days = [pd_day1, pd_day2]
pd_days_df = [day for day in pd_days if day != None]
pd_days_df = pd.DataFrame({'date':pd_days_df, 'class_type':'PD Day'})

#combine holidays and pddays into noclass df
no_class_df = pd.concat([holiday_df, pd_days_df]).reset_index(drop=True)
no_class_df.date = no_class_df.date.astype('datetime64[ns]')
no_class_df = no_class_df.sort_values('date')

#build df with all dates between start and end for this cohort
cohort_days = pd.DataFrame([start_date,end_date], columns=['date'])
cohort_days.date = cohort_days.astype('datetime64[ns]')
cohort_days = cohort_days.set_index('date').asfreq('D')
cohort_days = cohort_days.reset_index()
cohort_days['weekday'] = cohort_days.date.dt.day_name()

#merge no class days into cohort dates
cohort_days = cohort_days.merge(no_class_df, how='left', on='date')

#remove weekends
cohort_days = cohort_days [~cohort_days.weekday.isin(['Saturday','Sunday'])].copy()

#determine hours for each day
cohort_days['hours'] = np.where(cohort_days.class_type.notnull(), 0,
                                np.where(cohort_days.weekday == 'Monday', 6, 
                                         np.where(cohort_days.weekday == 'Thursday', 6.5, 7)))

#count how many class days there are
class_days_count = cohort_days [cohort_days.hours != 0].reset_index(drop=True)
class_days_count['classday_num'] = class_days_count.index + 1

#merge class day count with cohort days
cohort_days_all = cohort_days.merge(class_days_count, how='left', 
                                on=['date','class_type','weekday','hours'])

In [7]:
print(f'There are {cohort_days_all.hours.sum()} hours set for this cohort.')

There are 665.5 hours set for this cohort.


#### Verify that there are enough cohort days for all lessons

In [8]:
#pull lesson days
lesson_df = pd.read_csv(lesson_file)

#count the number of lesson days and the number of cohort days
num_lessons = lesson_df.delivery_day_num.nunique()
num_days = cohort_days_all.classday_num.nunique()

#calculate overlap
overlap = num_lessons - num_days

#result
if num_lessons > num_days:
    print('!***************************************************************************************!')
    print(f'There are {num_lessons} lesson days, but there are only {num_days} days in the cohort.')
    print(f'This is {overlap} extra lesson day(s).')
    print(f'Remove that many days from the {lesson_file} to accomodate all lessons and rerun script.')
    print('!***************************************************************************************!')
elif num_lessons < num_days:
    print('!***************************************************************************************!')
    print(f'There are only {num_lessons} lesson days, but there are {num_days} days in the cohort.')
    print(f'This is {overlap*-1} extra cohort day(s).')
    print(f'Add that many days to the {lesson_file} to fill out the schedule and rerun script.')
    print('!***************************************************************************************!')
else:
    print('The lesson schedule matches the length of the cohort. Proceed to build calendar')

The lesson schedule matches the length of the cohort. Proceed to build calendar


In [6]:
#merge lesson days with cohort days
cohort_days_almost = cohort_days_all.merge(lesson_df, how='left', 
                      left_on='classday_num',
                      right_on='delivery_day_num')

#fill non-delivery days with event
cohort_days_almost.delivery_name = cohort_days_almost.delivery_name.fillna(cohort_days_almost.class_type)

#reduce down to final two columns and export
cohort_days_final = cohort_days_almost[['delivery_name','date']]
cohort_days_final.columns = ['Subject', 'Start Date']
cohort_days_final

,Subject,Start Date
0,welcome L00 welcome day [instr],2023-10-03
1,welcome L01 syllabus review [instr],2023-10-03
2,welcome L01 environment setup [instr],2023-10-03
3,fundamentals L00 python 101 [students],2023-10-04
4,fundamentals L00 what is data science [instr],2023-10-04
...,...,...
230,capstone L11 capstone work [instr],2024-03-12
231,capstone L12 review panel I [instr],2024-03-13
232,capstone L13 review panel II [instr],2024-03-14
233,capstone L14 review panel III [instr],2024-03-15


#### Generate student holidays --> paste into syllabus
- Paste in numbers/google sheet and copy formatted version
- Paste into cohort syllabus

In [7]:
# Create first and last day rows as DFs to concat to holiday_df
firstday = pd.DataFrame({'class_type': 'First Day of Class', 'date': start_date}, index =[0])
lastday = pd.DataFrame({'class_type': 'Graduation/Capstone Presentations', 'date': end_date}, index =[0])

#convert to datetime fmt
firstday.date = firstday.date.astype('datetime64')
lastday.date = lastday.date.astype('datetime64')

#reorder holiday_df for current cohort dates
stuholiday_df = holiday_df[(holiday_df.date > start_date) & (holiday_df.date < end_date)][['class_type', 'date']]

#append "NO CLASS - " to student holidays
stuholiday_df['class_type'] = "NO CLASS – " + stuholiday_df['class_type']

# Concat all important dates to one df
stuholiday_df = pd.concat([firstday, stuholiday_df, lastday])

#rename column
stuholiday_df = stuholiday_df.rename(columns={'class_type':'event'})

In [8]:
#export
stuholiday_df.to_clipboard(index=False)

#### Generate student facing calendar --> paste in syllabus
- Paste in numbers/google sheet and copy formatted version
- Paste into cohort syllabus

In [9]:
df = cohort_days_almost.rename(columns={'delivery_name':'subject'}).copy()

#remove holidays
df = df [df.class_type.isnull()]

#remove tasks & bonus lessons
df = df [~df.subject.str.contains('task')]
df = df [~df.subject.str.contains('bonus')]
df = df [~df.subject.str.contains('kickoff')]

#remove extra pieces 
df.subject = df.subject.str.replace('\[.+\]','', regex=True)
df.subject = df.subject.str.replace('\(.+\)','', regex=True)

#consolidate project lessons & review panels
df.subject = df.subject.str.replace('project.*','project', regex=True)
df.subject = df.subject.str.replace('review panel.*','capstone work', regex=True)

#remove all reviews
df = df [~df.subject.str.contains('review')]

#remove duplicate lessons
df = df.drop_duplicates('subject')

#convert date to datetime
df.date = df.date.astype('datetime64')  

#get the monday of each week
def get_monday(start_date):
    new = start_date + timedelta(days=-start_date.day_of_week)
    return new
df['week'] = df.date.apply(get_monday)

#group lessons by each week
df = df.groupby('week').subject.apply(lambda x: '\n'.join(x.astype(str).str.strip())).reset_index()

In [10]:
df.to_clipboard(index=False)

#### Save cohort calendar --> upload to google calendar 


In [11]:
calendar_name = cohort_name + '_cal.csv'
cohort_days_final.to_csv(calendar_name)

![newcalendarinstructions](NewClassCalendar.png)